# Workshop 3 (Assegnato il 3-11-2017)

## Realizzazione in PyPlasm del modello di una chiesa disegnato da Leonardo da Vinci

L'approccio è il seguente: si realizza un quarto di chiesa e poi si replicano le parti restanti per comporre la chiesa finale essendo essa perfettamente simmetrica

![church1](church1.JPG) ![church2](church2.JPG)

In [1]:
from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.00579149717721 seconds


In [10]:
# Sfera dimezzata definisce una mezza sfera di raggio radius
def sferaDimezzata (radius):
        def GSPHERE0 (subds):
            N , M = subds
            domain = Hpc(Grid([N*[PI/N],M*[PI/M]]))
            domain = MAT([[1,0,0,0],[PI/2,1,0,0],[PI,0,1,0],[0,0,0,1]])(domain)
            fx  = lambda p: radius * math.cos(p[0]) * math.sin  (p[1])
            fy  = lambda p: radius * math.cos(p[0]) * math.cos (p[1])
            fz  = lambda p: radius * math.sin(p[0])
            ret=  GMAP([fx, fy, fz])(domain)
            return ret
        return GSPHERE0

In [11]:
def sferaDimezzataX():
    spessore = CUBOID([0.1,0.1,0.1]) # spessore usato per riempire la sfera dimezzata
    return JOIN([sferaDimezzata(1.1)([30,30]),spessore])

In [12]:
cilindro = CYLINDER([1,2.9])(30) # cilindro definito per realizare le 4 entrate della chiesa
anello = T(3)(2.7)(CYLINDER([1.1,0.1])(30)) # primo anello del tetto del cilindro
fregioCircolare = T(3)(2.91)(CYLINDER([1.2,0.1])(30)) #tetto del cilindro delle 4 entrate

# Funzione che definisce il portone con fregio superiore delle 4 entrate della chiesa
def traveEAnta():
    antaSinistraPortone = T([1])([0.2])(CUBOID([0.1,0.1,1.2]))
    travePortone = T([1,3])([-0.1,1.2])(CUBOID([0.4,0.1,0.1]))
    trave2Portone = T([1,3])([-0.2,1.3])(CUBOID([0.6,0.1,0.15]))
    tettoPortone =  R([1,3])(-PI/4)(T([1,3])([-0.2,1.3])(CUBOID([0.7,0.1,0.15])))
    return TOP([STRUCT([antaSinistraPortone,travePortone,trave2Portone]),tettoPortone])

# Definisco i 2 fregi rettangolari del portico
fregioSinistro = T([1,2,3])([0.4,-0.05,0.5])(R([1,2])(PI-10)(CUBOID([0.15,0.1,1.9])))
fregioDestro= T([1,2,3])([0.8,-0.4,0.5])(R([1,2])(PI-10)(CUBOID([0.15,0.1,1.9])))
fregioSinistroPortico = T(2)(0.95)(STRUCT([traveEAnta(),fregioSinistro,fregioDestro]))
fregioDestroPortico = T(2)(0.95)(STRUCT([T([1,2])([1.09,-0.95])(R([1,2])(PI/2)(traveEAnta())),fregioSinistro,fregioDestro]))

# Creo il portico di entrata della chiesa
porticoConFregioSinistro = STRUCT([cilindro,fregioSinistroPortico,anello,fregioCircolare])
porticoConFregioDestro = STRUCT([cilindro,fregioDestroPortico,anello,fregioCircolare])
porticoSenzaFregio = STRUCT([cilindro,anello,fregioCircolare])

In [13]:
# Definisco l'entrata della chiesa con la sua piccola cupola
quartoDiSfera = R([1,2])(-PI/4)(R([1,3])(PI/2)(S([1,2,3])([1,1,1])(sferaDimezzataX())))
cupolaConBase = STRUCT([porticoSenzaFregio,T(3)(3)(quartoDiSfera)])

Child 3


In [14]:
# Unisco tra di loro il corpo della chiesa con la cupola principale
def corpoECupola():
    colonnato = DIFFERENCE([CUBOID([1.1,1.1,5.6]),T([1,2,3])([0.18,1,0.1])(CUBOID([0.8,0.2,0.5])),T([1,2,3])([0.43,1,1])(CUBOID([0.3,0.2,3.5]))])
    colonnatoEcupola = TOP([colonnato,S([1,2,3])([0.3,0.3,0.3])(cupolaConBase)])
    corpoCentrale = STRUCT([T([1,2])([4,4])(colonnatoEcupola),CUBOID([5,5,5.5])])
    return STRUCT([corpoCentrale,T([1,2,3])([2.5,2.5,1])(S([1,2,3])([2,2,2])(cupolaConBase)),T([1,2,3])([2.5,2.5,9])(S([1,2,3])([0.3,0.3,0.3])(cupolaConBase))])

In [15]:
# Creo un quarto della chiesa finale
def quartoDiChiesa():
    cupolaConFregioSinistro = STRUCT([porticoConFregioSinistro,T(3)(3)(quartoDiSfera)])
    cupolaConFregioDestro = STRUCT([porticoConFregioDestro,T(3)(3)(quartoDiSfera)])
    return INTERSECTION([T([2])([-2.5])(CUBOID([4,4,10.5])),
                         STRUCT([cupolaConFregioSinistro,T([1,2])([2.5,-2.5])(cupolaConFregioDestro),
                                 T([1,2])([-2.5,-5])(corpoECupola())])])
    

In [16]:
# La chiesa finale è data dal quarto di chiesa ripetuto 4 volta, essendo essa perfettamente simmetrica
chiesaFinale = STRUCT([STRUCT([quartoDiChiesa(),T([1,2])([-2.5,-2.5])(R([1,2])(PI/2)(quartoDiChiesa()))]),(T(2)(-5)(R([1,2])(PI)(STRUCT([quartoDiChiesa(),T([1,2])([-2.5,-2.5])(R([1,2])(PI/2)(quartoDiChiesa()))]))))])

In [17]:
VIEW(COLOR([0.6,0.4,0.2])(chiesaFinale))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000F6B2030> >

![chiesafinale](chiesaFinale.JPG)